In [ ]:
import vk
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm, tqdm_pandas

In [ ]:
# opennig vk session with access token.
session = vk.Session(access_token='0dfed272d8c2c0882f26fabaee86c1c23eca901348c1e8476f93a1fd83fe06b4b9fc4070f7c0f1aa269f6')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

In [ ]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [ ]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

In [ ]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [ ]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

In [ ]:
def active_users_info(x):
    # x is api newsfeed search result
    # use each user only once to get personal info (different posts from same user have the same personal info)
    unique_users = list(set(x['owner_id'].astype(int)))
    users = pd.DataFrame(api.users.get(user_ids=unique_users,fields = ['occupation','education',
                                                                       'universities','bdate',
                                                                       'city','interests',
                                                                       'activities','site']))

    try:     
        # fill active users status with valid
        users['deactivated'].fillna('Valid',inplace = True) 
        # select all deactivated user ids
        deactivated_ids = list(users[users['deactivated']!='Valid'].uid)

        # select all active user posts
        active_users = x[-x.owner_id.isin(deactivated_ids)]
        active_users_info = users[users['deactivated']=='Valid']
    except:
        active_users = x  
        active_users_info = users

    # merge post info and user info
    overall_data = active_users.merge(active_users_info, left_on='owner_id', right_on='uid')
    return overall_data

In [ ]:
%%time
key_words = ['#глаза',
             '#глазки', '#зрение', '#лечениезрения', '#детскоезрение', '#лечениеглаз',
             '#офтальмолог','#офтальмология', '#офтальмохирург', '#офтальмохирургия', '#детскоезрение',
             '#детскийофтальмолог', '#офтальмологмосква','#дакриоцистит', '#дакриоциститноворожденных',
             '#дакриоциститмассаж','#дакриоциститлечение', '#дакриоцисториностомия', '#зондированиеносослёзногоканала',
             '#зондированиеноворождённых','#нистагм', '#лечениенистагма','#нистагмоперация', '#нистагмъ',
             '#близорукость', '#миопия', '#лечениеблизорукости', '#близорукостьнеприговор',
             '#близоркостьлечение', '#близорукостьонатакая', '#близорукостьнепорок',
             '#близорукостьудетей','#дальнозоркость', '#гиперметропия', '#лечениедальнозоркости',
             '#дальнозоркостьлечение','#дальнозоркостьудетей', '#дальнозоркости','#косоглазие',
             '#страбизм', '#лечениекосоглазия', '#косоглазиелечение', '#косоглазиеэтосексуально',
             '#косоглазиебишкек', '#косоглазиенепорок', '#косоглазие дети', '#косоглазиеэтомодно',
             '#астигматизм', '#астигматизмлечение', '#астигматизмудетей','#амблиопия', '#амблиопиявысокойстепени',
             '#амблиопиямосква', '#катаракта','#катарактавладивосток', '#катарактачелябинск',
             '#катаракта лечение москва','#катарактаоперация', '#глаукома', '#глаукомалечение', 
             '#глаукомавладивосток','#глаукомачелябинск', '#птоз', '#птозверхнеговека', '#птозвека',
             '#птозвек', '#птозанет','#лазернаякоррекция', '#лазернаякоррекциязрения', '#лазернаякоррекциязрениямосква',
             '#лазернаякоррекцияакция', '#лазернаякоррекциямосква', '#лазернаякоррекциязренияомикрон',
             '#лазернаякоррекцияскидки', '#lasik', '#фрк', '#ласик', '#ласикмосква', '#ячмень', '#халязион']
desired_time = 86400*365 # how much back we want to go in seconds
entire_df = pd.DataFrame()
# Overall iteration count
k = 1
for key_word in key_words:
    print('search for keyword: ', key_word)
    date = time.time() # now
    excess_time = True
    all_df = pd.DataFrame()
    # Iteration count for each key word
    c=1
    # Iterator for error count (TimeOut Error)
    error = 0
    while excess_time:
        # in case a ReadTimeOut Error occurs
        try:
            # Print which iteration it is for current key word   
            print(c,'iteration')

            '''
            # Resting each overall 60 iterations
            if k%60 == 0:
                print('Rest for 10 min')
                time.sleep(60*10)
                print('Work')
            '''

            # print iteration count, extended tells us group vs user
            newsfeed_search_results = api.newsfeed.search(q=[key_word],count=200, end_time = date-1, start_time=1,extended=1)
            nsr_df = pd.DataFrame(newsfeed_search_results[1:])
            # Filter posts done by groups
            try:
                nsr_df = nsr_df[nsr_df.owner_id>0].reset_index()
            except:
                pass
            
            if len(nsr_df)>0: # check if g contains data
                new_data = active_users_info(nsr_df)
                all_df = pd.concat([all_df,new_data])
                date=newsfeed_search_results[-1]['date']
                time.sleep(31) # Sleep time to not exceed requests of the same method

            elif len(nsr_df) <= 0 and c == 1: # check if there is data
                print('No posts with that keyword(s)!')
                time.sleep(31) # Sleep time to not exceed requests of the same method
                break

            elif len(nsr_df) <= 0 and c != 1: # check if there are posts eft to scrape
                print('No more posts to scrape!')
                time.sleep(31) # Sleep time to not exceed requests of the same method
                break

            # increment the iteration counts
            c+=1
            k+=1

            if date<time.time()-desired_time: # check termination condition
                excess_time = False
        # just repeat the request
        except:
            print('Timeout Error: sleep for 2 min')
            time.sleep(60*2)
            error += 1
            if error == 2:
                print('Encountered TimeOut Error 2 times')
                break
            else:
                continue
        
    all_df['keyword'] = key_word
    print('all_df length: ',len(all_df))
    entire_df = pd.concat([entire_df,all_df])
    

# getting list of cities by their ids
all_city_ids = entire_df.city.unique().astype(int)
city_name = []
# cant scrape more than 1000 in 1 request
for cities in range(int(len(all_city_ids)/1000)+1):
    city = api.database.getCitiesById(city_ids = all_city_ids[cities*1000:(cities+1)*1000])
    city_name = city_name+city

# creating dataframe from the list of the cities    
entire_df['city'] = entire_df['city'].astype(int)
# merging 2 dataframes on city id
entire_df = entire_df.merge(pd.DataFrame(city_name), left_on='city', right_on='cid')

print('entire_df length: ',len(entire_df))

In [ ]:
entire_df.to_csv(r'D:\vk_all_keywors_365_days_vk8_10Feb.csv')

In [ ]:
%%time
overall_data=entire_df.copy()
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['post_id','user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date','key_word',
                                 'city','university','graduation_year','faculty_name',
                                 'eduaction_status','activities','interests','work_type',
                                 'work_place','site','bdate'])
# resetting the index it make circles of 0-200
#active_users.reset_index(inplace = True)
    
# making function to apply to get platform
def platform(x):
    try:
        return x.post_source['type'] + '_' + x.post_source['platform']
    except:
        return x.post_source['type']

# making function to get work type    
def type_scrap(x):
    try:
        return x['type']
    except:
        return None
    
# making function to get work place   
def place_scrap(x):
    try:
        return x['name']
    except:
        return None

    
new_df['post_id'] = overall_data['id']
new_df['platform'] = overall_data.apply(platform, axis = 1)
new_df['key_word'] = overall_data['keyword']
new_df['user_id'] = overall_data['user'].apply(lambda x: x['uid'])   
new_df['Name_Surname'] = overall_data['user'].apply(lambda x: x['first_name'])+'_'+overall_data['user'].apply(lambda x: x['last_name'])
new_df['post'] = overall_data['text']
new_df['likes_count'] = overall_data['likes'].apply(lambda x: x['count'])
new_df['comments_count'] = overall_data['comments'].apply(lambda x: x['count']) 
new_df['reposts_count'] = overall_data['reposts'].apply(lambda x: x['count'])
new_df['date'] = overall_data['date'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
new_df['sex'] = overall_data['user'].apply(lambda x: 'Female' if x['sex']==1 else 'Male' if x['sex']==2  else 'Not Specified')
new_df['city'] = overall_data['name']
new_df['university'] = overall_data['university_name']
new_df['gradutaion_year'] = overall_data['graduation']#.astype(int) #some nan value occur
new_df['faculty_name'] = overall_data['faculty_name']
new_df['education_status'] = overall_data['education_status']
new_df['activities'] = overall_data['activities']
new_df['interests'] = overall_data['interests']
new_df['work_type'] = overall_data['occupation'].apply(type_scrap)
new_df['work_place'] = overall_data['occupation'].apply(place_scrap)
new_df['site'] = overall_data['site']
new_df['bdate'] = overall_data['bdate']
#new_df.head()

In [ ]:
new_df.to_csv(r'D:\vk_all_keywors_365_days_vk8_10Feb_pretty.csv')

In [ ]:
# getting the comments of posts
def commentscrapper(x):
    uid = x['user_id']
    pid = x['post_id']
    likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
    time.sleep(3)
    reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
    time.sleep(3)
    try:           
        coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
        time.sleep(3)
        return {'comments':coms[1:],'users_liked': likes,'users_reposted': reposts,'user_id': uid, 'post_id':pid}
    except:
        return {'comments':'Access Denied','users_liked':likes,'users_reposted': reposts,'user_id': uid, 'post_id':pid}
        time.sleep(3)  

In [ ]:
clr_data3 = new_df.head().copy()#pd.DataFrame()
clr_data3['comments'] = None
clr_data3['users_liked'] = None
clr_data3['users_reposted'] = None
for i in tqdm(range(new_df.head().shape[0])):
    clr = commentscrapper(new_df.iloc[i])
    clr_data3.set_value(i,'comments' ,clr['comments'] )     
    clr_data3.set_value(i,'users_liked', clr['users_liked'])          
    clr_data3.set_value(i,'users_reposted', clr['users_reposted'])    

#final_data = pd.concat([new_df,clr_data],axis = 1)

In [ ]:
clr_data.to_csv('clr_demo.csv')

In [ ]:
short_df = final_data.copy()

In [ ]:
pd.options.display.max_columns = 45
short_df.head()

In [ ]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.text.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [ ]:
not_long = short_df[short_df['post']!='Too Long'].reset_index().drop('index', axis = 1)

In [ ]:
import re
list_of_animals = 'кот|кошка|котёнок|котяра|котяры|котяр|котище|котик|собака|пес|собачка|кобель|псина|сука|птица|пташка|птенцы|птенец|щенок'
def find_animal(x):
    my_list = re.findall(list_of_animals,x['post'])
    if len(my_list) == 0:
        return(True)
    else:
        return(False)

In [ ]:
new_df['animal'] = new_df.apply(find_animal,axis = 1)

In [ ]:
(new_df.animal == True).value_counts()

In [ ]:
short_df.to_csv(r'D:\vk_all_keywords_365_days_short.csv')